## Load environment and check for folder

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import subprocess
import os
base_dir = "/content/gdrive/MyDrive/final-project-paralelizacao-tecnica-de-meios-tons"
get_ipython().run_line_magic('cd', base_dir)

# assert that we have the exercise folder
assert 'repositorio' in os.listdir(), "Exercise folder not found"

Cuda de Técnicas de de Meios-Tons

import os
import subprocess

base_dir = os.getcwd()
exercise_dir = os.path.join(base_dir, "repositorio")
build_dir = os.path.join(exercise_dir, 'build')

# Função para rodar comandos de shell
def run_command(command, work_dir):
    return subprocess.run(command, shell=True, cwd=work_dir, text=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Preparar build
run_command('cmake -E remove_directory build', exercise_dir)
run_command('cmake -E make_directory build', exercise_dir)
assert run_command('cmake ..', build_dir).returncode == 0

make_result = run_command('make -j', build_dir)
print("MAKE STDOUT:\n", make_result.stdout)
print("MAKE STDERR:\n", make_result.stderr)
assert make_result.returncode == 0, "Erro na compilação (make)"


# Lista de imagens de teste
test_images = [
    "img/city.png",
    "img/baboon_colorida.png",
    "img/monalisa.png",
    "img/seagull.png",
    "img/butterfly.png"
]

# Modos de dithering suportados
modos = ["FloydSteinberg", "StevensonArce", "Burkes", "Sierra", "Stucki", "JarvisJudiceNinke"]

# Executar os testes
for i, image in enumerate(test_images, start=1):
    for modo in modos:
        print(f"\n===== Teste {i} - Modo: {modo} - Imagem: {image} =====")

        exec_path = os.path.join(build_dir, 'cuda_dithering')
        img_path = os.path.join(exercise_dir, image)
        output_image = os.path.join(exercise_dir, 'out', f'teste_{i}_{modo}.png')
        time_file = os.path.join(build_dir, f"time_{i}_{modo}.txt")

        subprocess.run(f"chmod +x {exec_path}", shell=True)

        command = f"/usr/bin/time -f '%e' {exec_path} -i {img_path} -m {modo} -g > /dev/null 2> {time_file}"
        print(f"Running: {command}")
        run = subprocess.run(command, shell=True, cwd=exercise_dir)

        if os.path.exists(time_file):
            with open(time_file) as f:
                runtime = f.read().strip()
            print(f"Tempo de execução: {runtime} segundos")
        else:
            print(f"Falha ao medir tempo para {image} no modo {modo}")
